<a href="https://colab.research.google.com/github/HromovAndrey/II_mood_4_part_4/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%BD%D0%B0_%D1%80%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D1%96%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [9]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)


Дані про розпад протонів

[посилання](https://www.kaggle.com/datasets/fedesoriano/multijet-primary-dataset)

In [10]:
# трохи обробки даних
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/MultiJetRun2010B.csv')

df = df[df['nBJets'] != 2]

columns = df.columns.delete(-1)
data = df[columns]
df[columns] = (data - data.min()) / (data.max() - data.min())

Дані погано збалансовані

In [11]:
df['nBJets'].value_counts()

,count
nBJets,
0,20615
1,1056


In [12]:
df.head()

,Run,Lumi,Event,MR,Rsq,E1,Px1,Py1,Pz1,E2,Px2,Py2,Pz2,HT,MET,nJets,nBJets
0,0.082072,0.0,0.000002,0.077191,0.041860,0.044618,0.342725,0.531326,0.480732,0.055588,0.696681,0.457241,0.440224,0.061659,0.043047,0.0,0
1,0.082072,0.0,0.000006,0.052182,0.055190,0.018690,0.493614,0.585784,0.491979,0.017684,0.567978,0.430552,0.474613,0.025133,0.034687,0.0,0
2,0.082072,0.0,0.000015,0.154156,0.035263,0.101251,0.540018,0.671386,0.470196,0.273968,0.533484,0.338381,0.325311,0.165718,0.059381,0.2,0
3,0.082072,0.0,0.000048,0.106599,0.123328,0.063472,0.306290,0.523352,0.483465,0.039306,0.713922,0.422854,0.493248,0.101707,0.108566,0.0,0
4,0.082072,0.0,0.000063,0.072599,0.024611,0.383563,0.508826,0.564737,0.292545,0.224039,0.550834,0.433906,0.361549,0.110721,0.018941,0.2,0


Потрібно на основі інформації про розпад спрогнозувати наявність високоенергетичного випромінювання `nBJets`

# Завдання 1
Створіть клас Dataset для тренувальних та тестових даних

In [13]:
class ProtonDecayDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.FloatTensor(data.values)
        self.targets = torch.FloatTensor(targets.values).reshape(-1, 1)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.targets[index]

# Завдання 2
Створіть DataLoader

In [14]:
X = df.drop(columns=['nBJets'])
y = df['nBJets']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

train_dataset = ProtonDecayDataset(X_train, y_train)
test_dataset = ProtonDecayDataset(X_test, y_test)

batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Завдання 3
Створіть нейромережу

In [15]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.layer3(x))
        return x

input_dim = X_train.shape[1]
model = SimpleNN(input_dim).to(device)


# Завдання 4
Визначте функцію втрат та оптимізатор, можливо графік швидкості навчання

In [16]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


# Завдання 5
Натренуйте нейромережу.

In [17]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    scheduler.step()

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')


Epoch 1/20, Loss: 0.4540
Epoch 2/20, Loss: 0.2059
Epoch 3/20, Loss: 0.1974
Epoch 4/20, Loss: 0.1952
Epoch 5/20, Loss: 0.1946
Epoch 6/20, Loss: 0.1943
Epoch 7/20, Loss: 0.1943
Epoch 8/20, Loss: 0.1942
Epoch 9/20, Loss: 0.1942
Epoch 10/20, Loss: 0.1942
Epoch 11/20, Loss: 0.1942
Epoch 12/20, Loss: 0.1941
Epoch 13/20, Loss: 0.1941
Epoch 14/20, Loss: 0.1941
Epoch 15/20, Loss: 0.1941
Epoch 16/20, Loss: 0.1941
Epoch 17/20, Loss: 0.1941
Epoch 18/20, Loss: 0.1941
Epoch 19/20, Loss: 0.1941
Epoch 20/20, Loss: 0.1941


# Завдання 6
Збережіть нейромережу та зробіть прогноз

In [18]:
torch.save(model.state_dict(), 'proton_decay_model.pth')

model.eval()
with torch.no_grad():
    predictions = []
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())

predictions = [1 if x >= 0.5 else 0 for x in predictions]

from sklearn.metrics import accuracy_score, roc_auc_score

accuracy = accuracy_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, predictions)

print(f'Accuracy: {accuracy:.4f}, ROC AUC: {roc_auc:.4f}')

Accuracy: 0.9513, ROC AUC: 0.5000
